# Proyecto integrador

Miguel Ángel Pérez López A01750145

Ariadna Huesca Coronado A01749161

Lenguaje: GO

https://golang.org/ref/spec#Floating-point_literals

http://www.regular-expressions.info/unicode.html#prop

In [1]:
;; External function required for this notebook.
(require '[clojure.test :refer [is]])

nil

## Pruebas unitarias

### Variable

In [2]:
;;; Regular expression:
(def GO-variable #"(?xi)
                    [\p{L}_][\p{L}\p{N}_]*") 
(is (re-matches GO-variable "a"))
(is (re-matches GO-variable "_x9"))
(is (re-matches GO-variable "ThisVariableIsExported"))
(is (re-matches GO-variable "αβ"))
(is (not (re-matches GO-variable ".")))

true

STRING

In [3]:
(def GO-string #"(?xi)
                    `[^`]*`|
                    \"([^\\\"]|
                        \\ 
                        ([abfnrtv\\\"`]|
                          U[\dA-F]{4}|
                          [0-7]{3}|
                          x[\dA-F]{2}))* 
                    \"") 
(is (re-matches GO-string "`abc`"))
(is (re-matches GO-string "`\n \n`"))
(is (re-matches GO-string "\"\\\"\""))
(is (re-matches GO-string "\"Hello, world!\\n\""))
(is (re-matches GO-string "\"日本語\""))
(is (re-matches GO-string "\"\\u65e5本\\U00008a9e\""))
(is (re-matches GO-string "\"\\xff\\u00FF\""))
(is (not (re-matches GO-string "\"\\0\"")))
(is (not (re-matches GO-string "\"\\xa\"")));este era con U
(is (not (re-matches GO-string "\"\\UabcZ\"")));este era con U
(is (not (re-matches GO-string "\"\\\"")));este era con U

true

In [4]:
(def GO-string #"(?xi)
                    (import|var|true|false)\s") 
(is (re-matches GO-string "import "))
(is (re-matches GO-string "var"))
(is (re-matches GO-string "true"))
(is (re-matches GO-string "false"))
(is (not (re-matches GO-string " ")))
(is (not (re-matches GO-string "importar")))
(is (not (re-matches GO-string "_var")))


FAIL in () (NO_SOURCE_FILE:4)
expected: (re-matches GO-string "var")
  actual: (not (re-matches #"(?xi)
                    (import|var|true|false)\s" "var"))

FAIL in () (NO_SOURCE_FILE:5)
expected: (re-matches GO-string "true")
  actual: (not (re-matches #"(?xi)
                    (import|var|true|false)\s" "true"))

FAIL in () (NO_SOURCE_FILE:6)
expected: (re-matches GO-string "false")
  actual: (not (re-matches #"(?xi)
                    (import|var|true|false)\s" "false"))


true

1. fn2: exportar archivo html
2. fn1: (inicio intermedio final) : string 
3. intermedio: ("<span class=" " resultado(float,var) ">"  contenido "</span>"
4. contenido: (< (menor que)   ➔   &lt; > (mayor que)   ➔   &gt; & (ampersand)   ➔   &amp;)

## Secuencial

### Secuencia Inicial

In [5]:
(def regex #"(?xi)
  ((?:import|var|true|false|func|switch|if|return|case|for|new|Float|struct|type)[^A-Z\d]{1})  # Grupo 1: Constantes
  |( [\p{L}_][\p{L}\p{N}_]* )                # Grupo 2: Variables
  |((?:\d+(?:_\d+)*)*\.\d+(?:_\d+)*(?:[eE][+-]??\d+(?:_\d+)*)* #Grupo 3 Flotantes
    |(?:\d+(?:_\d+)*)+\.\d*(?:_\d+)*(?:[eE][+-]??\d+(?:\d+)*)*
    |0[xX][_]*(?:[0-9a-fA-F]+(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]*
    |[0-9a-fA-F]*(?:_[0-9a-fA-F]+)*[.]*[0-9a-fA-F]+)
    (?:_[0-9a-fA-F]+)*[pPeE][+-]??[0-9a-fA-F]+(?:_[0-9a-fA-F]+)*)
  | (0[oO][_]*[0-7]+(?:_[0-7]+)*              #Grupo 4 enteros
    |0[xX][0-9a-fA-F]+(?:_[0-9a-fA-F]+)*
    |0[bB][01]+(?:_[01]+)*|(?:[0]+\d*(?:_\d+)*
    |[1-9]+\d*(?:_\d+)*)[eE][+-]??(?:[0]+\d*(?:_\d+)*|[1-9]+\d*(?:_\d+)*)
    |[0]+\d*(?:_\d+)*\d+|[1-9]+\d*(?:_\d+)*|0)
  | ( // .*|[/][*][^*]*[*]+(?:[^*/][^*]*[*]+)*[/]) # Grupo 5: Comentario
  | (`[^`]*?`|\"(?:[^\\]|\\(?:[abfnrtv\\\"`]
    |[^\"\n\\]|U[\dA-F]{4}|[0-7]{3}|x[\dA-F]{2}))*?\")   # Grupo 6: String
  | ([-+|*&/%^:]??=|(?:<<|>>|&\^)=)                   # Grupo 7: Operador de Asignacion
  | ([|]{2}|&{2}|!)                           # Grupo 8: Operador Lógico
  | (>>|<<|(?:&\^)|[-+*/%&|^])                # Grupo 9: Operador Aritmético
  | ( [\(\)\{\}] )                            # Grupo 10: Puntuación
  | ( \s )                                    # Grupo 11: Otro
  | ( . )                                     # Grupo 12: Carácter inválido")

#'user/regex

In [6]:
(defn tokenize-file
  [file-name]
  (->> (re-seq regex (slurp (clojure.string/join ["archivos/" file-name])))
       (map (fn [match]
              (let [token (match 0)]
                (cond
                  (match 1) [token "Constante"]
                  (match 2) [token "Variable"]
                  (match 3) [token "Flotante"]
                  (match 4) [token "Entero"]
                  (match 5) [token "Comentario"]
                  (match 6) [token "String"]
                  (match 7) [token "Asignacion"]
                  (match 8) [token "Logico"]
                  (match 9) [token "Aritmetico"]
                  (match 10) [token "Puntuacion"]
                  (match 11) [token "Espacio"]
                  (match 12) [token "Invalido"]))))))

#'user/tokenize-file

### En HTML

In [7]:
(defn reemplaza [string]
  "Remplaza carácteres &,<,> por sus equivalentes en html."
  (->> (clojure.string/replace string #"&" "&amp")
       (map (fn [string] (clojure.string/replace string #"<" "&lt")))
       (map (fn [string] (clojure.string/replace string #">" "&gt")))
       (reduce str)))

#'user/reemplaza

In [8]:
(defn etiquetas [file-name]
    "Genera las etiquetas HTML."
  (->> (tokenize-file file-name)
       (map (fn [t] (if (= (t 1) "Espacio")
                      (t 0)
                      (clojure.string/join 
                       ["<span id = '" 
                        (t 1) 
                        "'>" 
                        (reemplaza (t 0)) 
                        "</span>"]))))
       (reduce str)))

#'user/etiquetas

In [9]:
(defn final [file-name]
  (clojure.string/join [(slurp "inicio.txt")
                        (etiquetas file-name)
                        (slurp "final.txt")]))

#'user/final

In [10]:
;funcion final
(defn procesa-texto [file-name]
  (spit (clojure.string/join ["Resultado/" file-name ".html"]) 
        (final (clojure.string/join [file-name ".go"]))))

#'user/procesa-texto

In [11]:
(defn get-file-info
  [dir]
  (map (fn [f]
         [(.getPath f) (.isFile f) (.getName f)]);obtiene 3 informaciones de cada archivo
       (file-seq (clojure.java.io/file dir))));   para usarlo en la función valid-file-names qu esta abajo

#'user/get-file-info

In [12]:
;Esta función sirve para obtener todos los nombres que se encuentren en la carpeta a analizar.
;Como en la función de procesa-texto se pide el nombre del archivo sin extensión,
;cree esta función
(defn valid-file-names
  [dir]                                        ;obtiene los nombres del directorio
  (->> (for [f (get-file-info dir) :when (f 1)];(f 1) es true o false si es archivo o no
            (f 2)) ;si (f 1) es verdadero se incluye el nombre del archivo
       (remove (fn [string1] (re-seq #"(?xi) (checkpoint|[.]css|[.]html)" string1)));quita archivos checkpoint que hace jupyterlab, archivos html y css
       (map (fn [string] (clojure.string/replace string #".go" "")));los archivos go le quita su extensión
       (map (fn [name1] (clojure.string/join [dir "/" name1])))));le agrega su extensión que se pone como parámetro dir
                                                    ;porque si sólo se queda el nombre, no va a encontrar el archivo en caso de que
                                                    ;el archivo este en otro directorio

#'user/valid-file-names

## Solución con una lista

In [13]:
(defn part-one2    ;esta función hace lo mismo pero recibe una lista de los nombres
  [directory]
  (map procesa-texto directory))

#'user/part-one2

In [20]:
(def archivos-prueba (map #(clojure.string/replace % #"archivos/" "") (valid-file-names "archivos")))
(println archivos-prueba)

(binary128 binary16_test calculator client config_test decoder_test fuzzy_decoder fuzzy_decoder_test hellopb jsoniter_array_test otroServer point_test response server singlelinkedlist stream_str struct_embedded_test struct_tags_test test_stale_repositories Ultimo Untitled.ipynb)


nil

In [15]:
(part-one2 archivos-prueba)

Error printing return value (FileNotFoundException) at java.io.FileInputStream/open0 (FileInputStream.java:-2).
archivos\Untitled.ipynb.go (El sistema no puede encontrar el archivo especificado)


class clojure.lang.ExceptionInfo: 

## Paralelo

In [16]:
;esta función hace que haga una secuencia 
(defn ranges-str
  [p strings]
  (partition p strings))

#'user/ranges-str

In [17]:
(defn parallel
  [p archivo]
  (->> (partition p archivo)
       (pmap 
         #(part-one2 %))))

#'user/parallel

## Pruebas

In [18]:
(doseq [i (range 5)]
  (time (part-one2 
         archivos-prueba)))

"Elapsed time: 0.0169 msecs"
"Elapsed time: 0.0021 msecs"
"Elapsed time: 0.0017 msecs"
"Elapsed time: 0.0013 msecs"
"Elapsed time: 0.0013 msecs"


nil

In [19]:
(doseq [i (range 5)]
  (time (parallel
         10 archivos-prueba)))

"Elapsed time: 1.8538 msecs"
"Elapsed time: 0.0707 msecs"
"Elapsed time: 0.0582 msecs"
"Elapsed time: 0.074 msecs"
"Elapsed time: 0.0762 msecs"


nil